In [33]:
include("MiniCollectiveSpins.jl")
using PyPlot
using Statistics
using JLD2
using OrdinaryDiffEq
import PhysicalConstants.CODATA2018: c_0
using Unitful
using ProgressMeter
using NonlinearSolve
using SteadyStateDiffEq 
using BenchmarkTools

In [ ]:
""" Prepare the initial vector u0 """
function u0_CFunction(phi_array, theta_array, op_list)
    u0 = ones(ComplexF64, length(op_list))
    for i in 1:length(op_list)
        if length(op_list[i]) == 1
            j = Int(op_list[i][1] % 10^floor(log10(abs(op_list[i][1]))-1)) # Atom nbr
            if parse(Int, string(op_list[i][1])[1:2]) == 22
                u0[i] = cos(theta_array[j]/2)^2
            elseif parse(Int, string(op_list[i][1])[1:2]) == 21
                u0[i] = cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
            else
                println(op_list[i][1])
            end
        end

        if length(op_list[i]) == 2
            for op in op_list[i]
                j = Int(op % 10^floor(log10(abs(op))-1)) # Atom nbr
                if parse(Int, string(op)[1:2]) == 22
                    u0[i] *= cos(theta_array[j]/2)^2
                elseif parse(Int, string(op)[1:2]) == 21
                    u0[i] *= cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
                elseif parse(Int, string(op)[1:2]) == 12
                    u0[i] *= cos(theta_array[j]/2)*exp(-1im*phi_array[j])*sin(theta_array[j]/2)
                else
                    println(op)
                end
            end
        end
    end
    return u0
end


""" Create a random distribution, save it, computes the corresponding parameters an return the stationnary state. 
If compute_t_evolution, compute the whole evolution, else only the stationnary state. """
function solve_random_distrib(chunk, f, op_list, N, n, d0_lb, threshold_std, nbr_std)
    popup_t, nbr_error_t, list_t, sol_t = [], [], [], []
    sol_ss, time_ss = [], []

    for i in chunk
        # Compute distribution
        L = (N/n)^(1/3) # Change the volume to keep the density cste
        r0 = [[rand(Float64)*L, rand(Float64)*L, rand(Float64)*L] for i in 1:N]

        # Choose a distribution where the minimum distance between the atoms is bigger than d0_min
        while min_r0(r0) < d0_lb
            r0 = [[rand(Float64)*L, rand(Float64)*L, rand(Float64)*L] for i in 1:N]
        end

        # Save the atoms position for comparison with QuantumOptics
        @save "r0/r0_N_$(N)_r_$i.jdl2" r0 L

        # Compute the parameters
        system = SpinCollection(r0, e, gammas=1.)
        Ω_CS = OmegaMatrix(system)
        Γ_CS = GammaMatrix(system)
        Γij_ = [Γ_CS[i, j] for i = 1:N for j=1:N]
        Ωij_ = [Ω_CS[i, j] for i = 1:N for j=1:N if i≠j]
        exp_RO_ = [exp(1im*r0[i]'kl) for i = N:-1:1] # We go in the decreasing direction to avoid exp_RO(10) being replace by exp_RO(1)0
        conj_exp_RO_ = [exp(-1im*r0[i]'kl) for i = N:-1:1]
        p0 = ComplexF64.([Γij_; Ωij_; exp_RO_; conj_exp_RO_; Ω_RO/2])
        
        # Load the functions
        fsolve(du, u, p, t) = f(du, u, p0)

        phi_array_0, theta_array_0 = zeros(N), ones(N)*π # We start from all the atoms in the GS
        u0 = u0_CFunction(phi_array_0, theta_array_0, op_list)

        function condition(u, t, integrator)
            if integrator.iter > nbr_std+1
                return maximum([real(mean([integrator.sol.u[j][i] for i = 1:N])) for j = length(integrator.sol.u)-nbr_std:length(integrator.sol.u)])-minimum([real(mean([integrator.sol.u[j][i] for i = 1:N])) for j = length(integrator.sol.u)-nbr_std:length(integrator.sol.u)]) < threshold_std
            else
                return false
            end
        end

        function affect!(integrator)
            push!(sol_ss, sum(real(copy(integrator.u)[1:N])))
            push!(time_ss, integrator.t)
        end

        cb = DiscreteCallback(condition, affect!)

        prob = OrdinaryDiffEq.ODEProblem(fsolve, u0, (0, t_end))
        
        println("$i, $chunk")

        sol = OrdinaryDiffEq.solve(prob, OrdinaryDiffEq.DP5();
                    reltol=1.0e-5,
                    abstol=1.0e-5,
                    callback = cb)

        if SciMLBase.successful_retcode(sol)
            push!(list_t, sol.t)
            push!(popup_t, [sum(real(sol.u[i][1:N])) for i=1:length(sol.t)])
            push!(sol_t, sol.u)
        else
            push!(nbr_error_t, i)
        end
    end
    return popup_t, nbr_error_t, list_t, sol_t, sol_ss, time_ss
end

""" Return the minimum distance of a distribution of atoms r0 """
function min_r0(r0)
    N = length(r0)
    d0 = zeros(N, N) # Repetiton, atom i, distance from atom j
    for j in 1:N
        for k = 1:N
            d0[j, k] = norm(r0[j]-r0[k])
        end
    end
    return minimum(d0[d0 .> 0])
end

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file://c:/Users/Joschka/Desktop/Remy/Codes/SR_droplets/QuantumCumulants/Model9/In[34]#72:275\[90mIn[34]:72:275[0;0m]8;;\
            if integrator.iter > nbr_std+1
                return maximum([real(mean([integrator.sol.u[j][i] for i = 1:N])) for j = length(integrator.sol.u)-nbr_std:length(integrator.sol.u)])-maximum([real(mean([integrator.sol.u[j][i] for i = 1:N])) for j = length(integrator.sol.u)-nbr_std:length(integrator.sol.u)])[48;2;120;70;70m[0;0m) < threshold_std
[90m#                                                                                                                                                                                                                                                                                 └ ── [0;0m[91mExpected `end`[0;0m

### Define the system

In [ ]:
# Nbr of particles
N = 4
r = 10 # Nbr of repetitions

# Normalisation parameters
λ = 421e-9
γ = 32.7e6 # In Hz

# Physical values
ω0 = (2π*ustrip(c_0)/λ)
ωl = ω0
kl = [ustrip(c_0)/ωl, 0, 0] # Laser along x
Ω_RO = 1e7 # Taken from Barbut arXiv:2412.02541v1

# Fixed density
n0 = 100 #2e3 # atoms per unit of volume (already normalized)
d0_lb = 1e-10 # Minimum distance between the atoms (lower boundary) in m

# Normalization
ω0 = ω0 / γ
ωl = ωl / γ
kl = kl * λ
Ω_RO = Ω_RO / γ
d0_lb = d0_lb / λ

# Quantization axis along z
e = [0, 0, 1.]

# Integration parameter
t_end = 1e2
threshold_std, nbr_std = 1e-3, 100;

### Compute stationnary state for r repetitions

In [ ]:
# Create the directories
if !isdir("r0")
    mkdir("r0")
end
if !isdir("Images_distribution")
    mkdir("Images_distribution")
end
if !isdir("solutions")
    mkdir("solutions")
end
nothing 

In [ ]:
# Prepare the function
libpath ="libs/liballfuncs_$N.dll"
f(du, u, params) = ccall(("diffeqf", libpath), Cvoid, (Ptr{ComplexF64}, Ptr{ComplexF64}, Ptr{ComplexF64}), du, u, params)

f (generic function with 1 method)

In [ ]:
@load "op_list/op_list_$N.jdl2" op_list
list_r = 1:r
chunks = Iterators.partition(list_r, cld(length(list_r), Threads.nthreads()))

tasks = map(chunks) do chunk # Split the different distributions into chuncks solved on each core
    Threads.@spawn solve_random_distrib(chunk, f, op_list, N, n0, d0_lb, threshold_std, nbr_std)
end

# Gather the data from the different threads
sol_tasks = fetch.(tasks)
popup_t, nbr_error_t, list_t, sol_t, sol_ss, time_ss = [vcat([s[i] for s in sol_tasks]...) for i = 1:6]
println("Nbr errors t_evol = "*string(length(nbr_error_t)))

1, 1:2
9, 9:10
7, 7:8
3, 3:4
5, 5:6
2, 1:2


LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching isless(::ComplexF64, ::ComplexF64)
    The function `isless` exists, but no method is defined for this combination of argument types.
    
    [0mClosest candidates are:
    [0m  isless([91m::Missing[39m, ::Any)
    [0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:87[24m[39m
    [0m  isless([91m::PyCall.PyObject[39m, ::Any)
    [0m[90m   @[39m [35mPyCall[39m [90mC:\Users\Joschka\.julia\packages\PyCall\1gn3u\src\[39m[90m[4mpyoperators.jl:75[24m[39m
    [0m  isless([91m::Unitful.AbstractQuantity[39m, ::Number)
    [0m[90m   @[39m [36mUnitful[39m [90mC:\Users\Joschka\.julia\packages\Unitful\M33vg\src\[39m[90m[4mquantities.jl:238[24m[39m
    [0m  ...
    
    Stacktrace:
      [1] [0m[1mmax[22m[0m[1m([22m[90mx[39m::[0mComplexF64, [90my[39m::[0mComplexF64[0m[1m)[22m
    [90m    @[39m [90mBase[39m [90m.\[39m[90m[4moperators.jl:527[24

In [ ]:
@save "solutions/sol_N_$(N)_r_$(r)" popup_t nbr_error_t list_t sol_t

In [ ]:
# @load "solutions/sol_N_$(N)_r_$(r)" popup_t nbr_error_t list_t sol_t

# Plots with time evolution

In [ ]:
fig, ax = subplots()
for i in 1:length(popup_t)
    line, = ax.plot(list_t[i], popup_t[i]) #, marker="o", markersize=3
end

for i in 1:length(time_ss)
    ax.scatter(time_ss[i], sol_ss[i], color="red")
end

ax.set_xlabel(L"$\gamma t$")
ax.set_ylabel(L"$\langle  n_{\uparrow} \rangle $")

suptitle("N = $N, r = $r, Starting from "*L"$|\downarrow \downarrow \rangle $")
pygui(false); gcf();
# pygui(true); show()